In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import smallgraph as sg
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount
import ubergauss.tools as ut
from yoda import graphs as ygraphs
import seaborn as sns
import pandas as pd
import needle

In [ ]:

import matplotlib as mpl
mpl.rc("figure", dpi=300)

In [ ]:
%%time
a,l = load_rfam(full= True, add_cov= False)

import yoda.alignments.clans as cl 

In [ ]:
rf15Labels = cl.getlabels_rfam15(a)

In [ ]:
# this is just to collect data to report in the paper
# asd= cl.get_label_dictionary(cl.clans_rfam15)
asd= cl.get_label_dictionary(cl.clans)
len(np.unique(list(asd.values()))) # 146
len(asd) # 463
#len(a)

In [ ]:
%%time
def atograph(ali):
    graph = ali2graph.set_weight_label(ali)
    graph = ali2graph.dillute(graph)
    graph = ali2graph.set_weight(graph)
    graph =   ali2graph.multiGraph(ali)
    graph =  ali2graph.donest(graph)
    return graph

graphss = ut.xxmap(atograph, a)

matrix = ygraphs.vectorize_graphs(graphss, normalization=True,
                      min_r = 2,min_d=1,
                      inner_normalization=True)

# sml.knn_accuracy(matrix,l)

In [ ]:
for i,e in enumerate(graphss):
    if not e:
        print(i)
        print(a[i].alignment.shape)


In [ ]:
a[1200].__dict__

In [ ]:
matrix.shape

In [ ]:
%%time
from  sklearn.neighbors import KNeighborsClassifier
from kiez import Kiez


ranks = needle.getranks(matrix,l)

In [ ]:
%%time
if False:
    def calcranks(matrix,l):
        knn = KNeighborsClassifier(n_neighbors=100)
        yy= np.array(l)
        knn.fit(matrix,yy)
        _, indices = knn.kneighbors(matrix)
        neighbor_labels = yy[indices]
        neighbor_labels = neighbor_labels[:, 1:]
        def searchrank(i,l):
            row = neighbor_labels[i] 
            indices = np.where(row == l)[0]
            return indices[0] if len(indices)>0 else 999999
        ranks = [searchrank(i,lab) for i,lab in enumerate(yy) if lab != 0]
        return np.array(ranks)



a2,l2 = load_rfam(full= True, add_cov= False)
mat = ygraphs.vectorize_alignments(a)
#mat = ygraphs.vectorize_alignments(a, min_r = 2, min_d=1)
ranks2 = needle.getranks(mat,l2)

In [ ]:
# PLOT importance weighted vectorization  vs normal vectorization , do we have at least 1 NN in the closest X instances?
sns.set_theme()
sns.set_context("talk")

#plt.plot([sum(ranks < x)/sum(l != 0) for x in range(1,50)])
#plt.plot([sum(ranks2 < x)/sum(l2 != 0) for x in range(1,50)])
y_label = 'Label Hit Rate'

meins = [sum(ranks < x)/sum(l != 0) for x in range(1,50)]
data = {y_label: meins, 'neighbors':lmz.Range(1,50), 'method' : 'KRAID'}

eden = [sum(ranks2 < x)/sum(l2 != 0) for x in range(1,50)]
data2 = {y_label: eden, 'neighbors':lmz.Range(1,50),'method' : 'NSPDK'}


df= pd.concat([pd.DataFrame(data),pd.DataFrame(data2)])
ax= sns.lineplot(df, x= 'neighbors', y= y_label, hue = 'method') 
plt.xlabel('neighbors')
plt.title('Hit Rate with full Rfam backdrop')


In [ ]:
# are there different successrates for more populated families? 
manyseq, fewseq = filter_by_seqcount(a)

def pl(x,l):
    x.append(0)
    mask = [ ll in x for ll in l ]
    ranks = needle.getranks(matrix[mask], l[mask])
    
    # ranks = calcranks(mat[mask], l[mask])
    data = [sum(ranks < xx)/sum(l[mask] != 0) for xx in range(1,21)]
    # plt.plot(data)
    return data


many =  pl(manyseq,l) #[sum(ranks < x)/sum(l != 0) for x in range(1,50)]
few = pl(fewseq,l) # [sum(ranks2 < x)/sum(l2 != 0) for x in range(1,50)]

y_label = 'Label Hit Rate'
set_label = 'sequence count'
repeats = 3
rand = [pl(needle.randomhalf(l),l) for x in range (repeats)]
randnei = lmz.Range(1,21)*repeats

data = {y_label: many, 'neighbors':lmz.Range(1,21), set_label  : 'high'}
data2 = {y_label: few, 'neighbors':lmz.Range(1,21),set_label : 'low'}
data3 = {y_label: lmz.Flatten(rand), 'neighbors':randnei,set_label : 'random split ±σ'} ## !!!!

data4 = {y_label: pl(list(np.unique(rf15Labels)), rf15Labels), 'neighbors':lmz.Range(1,21),set_label : 'rf15'}

df= pd.concat([pd.DataFrame(data),pd.DataFrame(data2), pd.DataFrame(data3), pd.DataFrame(data4)])

ax= sns.lineplot(df, x= 'neighbors', y= y_label, hue = set_label, errorbar='sd') 
plt.xlabel('neighbors')
plt.title('Hit Rate with full Rfam backdrop\nperformance by number of sequences')

In [ ]:
# quick hack to compare the test set to the rest
from sklearn import metrics
pdist  = metrics.pairwise_distances(matrix)
print(sml.average_precision_nozero(pdist,l))
print(sml.average_precision_nozero(pdist,rf15Labels))


In [ ]:
# if we know the neighbor, can we find the whole familiy? 

if False:
    knn = KNeighborsClassifier(n_neighbors=100)
    yy= np.array(l)
    knn.fit(matrix,yy)
    _, indices = knn.kneighbors(matrix[yy!=0])


# are two enough to find the rest? 

In [ ]:
# get the labels where we have more than 2 examples
needlelabels = needle.threeinstances(l)
# for each row (with the right label) we find the closest other instance according to the kernel
sim_label_idx = needle.sim_label_idx(matrix,l,needlelabels)

sim_label_idx

In [ ]:
needleranks = needle.getranks_mix(sim_label_idx,matrix)

In [ ]:
plt.plot( [sum(needleranks < x)/len(needleranks) for x in range(1,10)])
plt.title('this is like the clansearch, but we use the average vector to find the 3rd instance')

In [ ]:
sim_label_idx = needle.sim_label_idx_limited(needlelabels, l,matrix,maxrank=10)
needleranks = needle.getranks_mix(sim_label_idx,matrix)

In [ ]:
moreranks = needle.getranks_idx3(matrix,l,set([e[-1][2] for e in sim_label_idx]))
one = [sum(moreranks < x)/len(moreranks) for x in range(1,50)]
two=  [sum(needleranks < x)/len(needleranks) for x in range(1,50)]


data = {'Label Hit Rate': one, 'neighbors':lmz.Range(2,len(one)+2), 'Experiment' : 'distance matrix'}
data2 = {'Label Hit Rate': two, 'neighbors':lmz.Range(2,len(one)+2),'Experiment' : 'linear combination search'}
df= pd.concat([pd.DataFrame(data),pd.DataFrame(data2)])
title = 'Finding additional alignments for a clan'

ax= sns.lineplot(df, x= 'neighbors', y= 'Label Hit Rate', hue = 'Experiment') 
plt.ylabel('Label Hit Rate (≥ 2 Hits)')
plt.title(title)


In [ ]:
cm_dict_ab={}
cm_dict_ba={}
with open('rfam-weakpairs','r') as f:
    for line in f:
        if line:
            sline = line.split(' ')
            cm_dict_ab[sline[0]+sline[1]] = line
            cm_dict_ba[sline[1]+sline[0]] = line
            

In [ ]:
matrix

In [ ]:
vec = eg.Vectorizer( min_r = 2,min_d=1)
import structout.intlistV2 as so 

def nice(values, colorids=[]):
    symbols = '▁▂▃▄▅▆▇█'
    values = np.array(values)
    values = so.binning(values, count = len(symbols), ylim=False)
    color = lambda x: '\x1b[1;3%d;48m%s\x1b[0m' % (1, x)
    if not colorids: 
        return ''.join([symbols[v] for v in values])
    else:
        return ''.join([symbols[v] if i not in colorids else color(symbols[v]) for i,v in enumerate(values)])


r = []
done = set()
needles = []
rowz = []
for i,ind in enumerate(neigh_ind):
    # we want r0 and r1 to be equal and >0.. 
    rowlabels = yy[ind]
    if rowlabels[0] == 0 or rowlabels[0]!=rowlabels[1]:
       continue 
    # we want 
    c,d = ind[0], ind[1]
    if (c,d) in done:
        continue
    else:
        done.add((d,c))
    
    needle = np.array([ (a + b) /2  for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0])])
    needles.append(needle)
    rowz.append((i,c,d))
    #input = np.log1p(matrix[[c,d],:])
    #needle = np.asarray(np.sum(input, axis = 0))[0]
    #input = matrix[[c,d],:]
    #needle = np.asarray(np.sum(input, axis = 0))[0]
    # FAB
    #index_helper = np.array([((a+b)/2) if a and b else 0 for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0]) ])
    # needle = np.array([(a+b)/2 for a,b in zip(matrix[c].toarray()[0],matrix[d].toarray()[0]) if a and b])
    #indices = np.argsort(-np.dot(matrix.todense()[:, index_helper > 0] , needle.T )).A1 [:50]
    
    # dist, indices = k_inst.kneighbors([needle])
    
k_inst = Kiez(algorithm='SklearnNN', hubness='csls',  n_candidates = 100,  
              algorithm_kwargs= {'metric' : 'cosine'})
k_inst.fit(np.array(needles),matrix.toarray() )
dist, indices2 = k_inst.kneighbors()

for indices,rid in zip( indices2, rowz):
    i,c,d = rid
    # CLASS IDS 
    r.append(str(yy[indices]))
    print(r[-1])
    # RFAM IDS 
    # IMPORTANCES
    for i in indices[:10]: # FIST 20 HITS 
        graph = graphss[i]
        seqid_scr = {}
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            seqid_scr[n]= (ve*needle)[0]
        score_per_nucleotide = [ seqid_scr.get(j,0) for j in range(a[i].alignment.shape[1])]


        conserved = [ nid for nid,di in graphss[i].nodes(data=True) if di['weight']> .5]
        #### print per entry... 
        # NAME,  score 
        # position score + highlight 
        # structure line 
        # 5 ali lines 
        # newline 
        r.append(f"{a[i].gf['DE']} {a[i].gf['ID']}") # ! score later 
        # print(r[-1])
        r.append(f"{nice(score_per_nucleotide, conserved)}") 
        # print(r[-1])
        r.append(f"{a[i].gc['SS_cons']}") 
        # print(r[-1])
        for row in a[i].alignment[:5]:
            r.append(f'{"".join(row)}') 
        # print(r[-1])

        labels = a[c].label, a[i].label
        k = '%s.cm%s.cm' % labels
        r.append(cm_dict_ab.get(k,'None'))
        r.append(cm_dict_ba.get(k,'None'))
        labels = a[d].label, a[i].label
        k = '%s.cm%s.cm' % labels
        r.append(cm_dict_ab.get(k,'None'))
        r.append(cm_dict_ba.get(k,'None'))
        r.append(f'') 
        
    def id_to_gr(id):
        graph = graphs[id]
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            graph.nodes[n]['asd']= ve*needle
        return graph
    #gra = list(map( id_to_gr, indices[0][:5]))
    # ed.draw_graph_row(gra, vertex_color = 'asd') 
    def draw(g):
        if len(g) < 400: 
            ed.draw_graph(g, vertex_color = 'asd', secondary_vertex_color = 'weight') ; plt.show()
        else:
            print("TOO LARGE TO DRAW")
    # [draw(g) for g in gra]




In [ ]:
r

In [ ]:
np.array(needles).shape

In [ ]:
matrix.toarray().shape

In [ ]:
r[2]

In [ ]:
r = [e if e is not None else "None" for e in r]

In [ ]:
with open('data.txt','w') as file:
    file.write('\n'.join(r))
!cat data.txt | aha > data.html

In [ ]:
import sklearn
sklearn.__file__

In [ ]:
# CAN WE FIND NEW INSTANCES OF A CLUSTER IF WE PROVIDE THE WHOLE CLUSTER?

knn = KNeighborsClassifier(n_neighbors=100)
yy= np.array(l)
knn.fit(matrix ,yy)

y_needle =  {}

for y in np.unique(yy): 
    if y > 0:
        # pairwise search ... 
        indices = np.where (yy==y)[0]
        for a,b in [(i,j) for enu,i in enumerate(indices) for j in indices[enu+1:]]:
            input = np.log1p(matrix[[c,d],:])
            needle = np.asarray(np.sum(input, axis = 0))[0]           
        
    
        
        # input = matrix[yy == y]
        # needle = np.asarray(np.all((input > 0).todense(), axis = 0))
        
        dist, indices = knn.kneighbors(needle)
        print(y)
        print(yy[indices])
        print(dist)
        y_needle[y] = (needle, indices) 


In [ ]:
def show(y):
    needle, indices = y_needle[y]
    def id_to_gr(id):
        graph = graphs[id]
        for n,ve in zip(graph.nodes, vec.vertex_transform([graph])[0]):
            graph.nodes[n]['asd']= ve*needle[0]
        return graph
    gra = list(map( id_to_gr, indices[0][:5]))
    # ed.draw_graph_row(gra, vertex_color = 'asd') 
    
    def draw(g):
        if len(g) < 300: 
            ed.draw_graph(g, vertex_color = 'asd', secondary_vertex_color = 'weight') ; plt.show()
        else:
            print("TOO LARGE TO DRAW")
    [draw(g) for g in gra]


show(8)


In [ ]:
>>> rng = np.random.RandomState(0)
>>> source = rng.rand(61,65537)
>>> target = rng.rand(4108,65537)


# Step 2: Create a Kiez object
kiezz = Kiez(hubness='localscaling')  # 'ls' stands for Local Scaling, you can also use 'mp' for Mutual Proximity

# Step 3: Fit the Kiez object to your data
kiezz.fit(source)

# Step 4: Transform the data to get hubness-corrected distances
X_corrected = kiez.transform(source)

# Step 5: Convert the corrected data to a distance matrix
#distance_matrix = nn.kneighbors_graph(X_corrected, mode='distance').toarray()

In [ ]:
nn_ind.shap

In [ ]:
import numpy as np

a = np.arange(10)

a+=4

np.where(a < 6)[0]